In [1]:
import sympy as sp

from IPython.display import display

In [2]:
global T_ipj, T_ij, T_imj, T_ijp, T_ijm, delta_r, delta_0, r_ij, sig_a, sig_b 

In [3]:
T_ipj, T_ij, T_imj, T_ijp, T_ijm, delta_r, delta_0, r_ij, sig_a, sig_b  = sp.symbols([
    "T_{i+1\,j}",
    "T_{i\,j}",
    "T_{i-1\,j}",
    "T_{i\,j+1}",
    "T_{i\,j-1}",
    "\Delta_r",
    "\Delta_\\phi",
    "r_{ij}",
    "\sigma_A",
    "\sigma_B"
    ], real=True
)

In [ ]:
def subs_in_edp(d2Vdr2, dVdr, d2Vd02):
    return d2Vdr2 + 1/r_ij * dVdr + 1/r_ij**2 * d2Vd02

def second_central_r():
    return (T_ipj - 2 * T_ij + T_imj) / delta_r ** 2

def first_central_r():
    return (T_ipj - T_imj) / (2*delta_r)

def second_central_0():
    return (T_ijp - 2 * T_ij + T_ijm) / delta_0 ** 2

def taylor_r(material='', which='prog'):
    assert which in ['reg', 'prog']
    # reg:  retorna o valor da expansão pro ponto i-1
    # prog: retorna o valor da expansão pro ponto i+1
    dVdr = sp.symbols("\\frac{\partial\ V}{\\partial\ r}\Bigr|^{" + material + "}_{i\,j}", real=True)
    d2Vdr2 = sp.symbols("\\frac{\partial^2\ V}{\\partial\ r^2}\Bigr|^{" + material + "}_{i\,j}", real=True)

    signal = 1 if which == 'prog' else -1

    return (T_ij + signal * delta_r * dVdr + delta_r**2/2 * d2Vdr2, dVdr, d2Vdr2)

def taylor_0(material='', which='prog'):
    assert which in ['reg', 'prog']
    # reg:  retorna o valor da expansão pro ponto j-1
    # prog: retorna o valor da expansão pro ponto j+1
    dVd0 = sp.symbols("\\frac{\partial\ V}{\\partial\ \phi}\Bigr|^{" + material + "}_{i\,j}", real=True)
    d2Vd02 = sp.symbols("\\frac{\partial^2\ V}{\\partial\ \phi^2}\Bigr|^{" + material + "}_{i\,j}", real=True)

    signal = 1 if which == 'prog' else -1

    return (T_ij + signal * delta_0 * dVd0 + delta_0**2/2 * d2Vd02, dVd0, d2Vd02)

<img src="https://i.ibb.co/hRNQn2y/contornos.png" width="500px"/>

# Vermelho

In [ ]:
# Dado no enunciado

# Cinza

Apenas desenvolver a EDP:

1. Segunda diferença central em $r$
2. Primeira diferença central em $r$
3. Segunda diferença central em $\phi$

In [ ]:
edp = subs_in_edp(second_central_r(), first_central_r(), second_central_0())

eqn = sp.Eq(edp, 0)

In [ ]:
solutions = sp.solve(eqn, T_ij)

assert len(solutions) == 1

sp.collect(sp.factor(solutions[0]), [T_ipj, T_imj, T_ijp, T_ijm])

(T_{i+1,j}*(\Delta_\phi**2*\Delta_r*r_{ij} + 2*\Delta_\phi**2*r_{ij}**2) + 2*T_{i,j+1}*\Delta_r**2 + 2*T_{i,j-1}*\Delta_r**2 + T_{i-1,j}*(-\Delta_\phi**2*\Delta_r*r_{ij} + 2*\Delta_\phi**2*r_{ij}**2))/(4*(\Delta_\phi**2*r_{ij}**2 + \Delta_r**2))

# Azul

Há troca de meio, portanto deve-se:

1. Expandir por Taylor regressivo até a segunda ordem (como se estivesse em um ponto de extremidade), para o material A

2. Repetir o processo para o material B, mas com Taylor progressivo

3. Substituir a segunda derivada parcial na EDP para cada caso

4. As derivadas primeiras nas EDPs de cada caso serão desconhecidas, então igualam-se as duas equações por meio do critério de fluxo continuado

In [ ]:
# Desenvolvimento em A

taylorA, dVdrA, d2Vdr2A = taylor_r('A', 'reg')

solutionsA = sp.solve(sp.Eq(taylorA, T_imj), d2Vdr2A)

assert len(solutionsA) == 1

edpA = subs_in_edp(solutionsA[0], dVdrA, second_central_0())

dVdrA_solution = sp.solve(sp.Eq(edpA, 0), dVdrA)

assert len(dVdrA_solution) == 1

dVdrA_value = dVdrA_solution[0]

In [ ]:
# Desenvolvimento em B

taylorB, dVdrB, d2Vdr2B = taylor_r('B', 'prog')

solutionsB = sp.solve(sp.Eq(taylorB, T_ipj), d2Vdr2B)

assert len(solutionsB) == 1

edpB = subs_in_edp(solutionsB[0], dVdrB, second_central_0())

dVdrB_solution = sp.solve(sp.Eq(edpB, 0), dVdrB)

assert len(dVdrB_solution) == 1

dVdrB_value = dVdrB_solution[0]

In [ ]:
# Fluxo continuado

solutions = sp.solve(sp.Eq(dVdrA_value * sig_a, dVdrB_value * sig_b), T_ij)

assert len(solutions) == 1

sp.collect(sp.factor(solutions[0]), [T_ipj, T_imj, T_ijp, T_ijm])

-(T_{i+1,j}*(2*\Delta_\phi**2*\Delta_r*\sigma_B*r_{ij}**2 + 4*\Delta_\phi**2*\sigma_B*r_{ij}**3) + T_{i,j+1}*(-\Delta_r**3*\sigma_A + \Delta_r**3*\sigma_B + 2*\Delta_r**2*\sigma_A*r_{ij} + 2*\Delta_r**2*\sigma_B*r_{ij}) + T_{i,j-1}*(-\Delta_r**3*\sigma_A + \Delta_r**3*\sigma_B + 2*\Delta_r**2*\sigma_A*r_{ij} + 2*\Delta_r**2*\sigma_B*r_{ij}) + T_{i-1,j}*(-2*\Delta_\phi**2*\Delta_r*\sigma_A*r_{ij}**2 + 4*\Delta_\phi**2*\sigma_A*r_{ij}**3))/(2*(\Delta_\phi**2*r_{ij}**2 + \Delta_r**2)*(\Delta_r*\sigma_A - \Delta_r*\sigma_B - 2*\sigma_A*r_{ij} - 2*\sigma_B*r_{ij}))

# Verde

Análogo ao anterior:

1. Taylor regressivo em B

2. Taylor progressivo em A

3. Substituir e igualar equações

In [ ]:
# Desenvolvimento em B

taylorB, dVdrB, d2Vdr2B = taylor_r('B', 'reg')

solutionsB = sp.solve(sp.Eq(taylorB, T_imj), d2Vdr2B)

assert len(solutionsB) == 1

edpB = subs_in_edp(solutionsB[0], dVdrB, second_central_0())

dVdrB_solution = sp.solve(sp.Eq(edpB, 0), dVdrB)

assert len(dVdrB_solution) == 1

dVdrB_value = dVdrB_solution[0]

In [ ]:
# Desenvolvimento em A

taylorA, dVdrA, d2Vdr2A = taylor_r('A', 'prog')

solutionsA = sp.solve(sp.Eq(taylorA, T_ipj), d2Vdr2A)

assert len(solutionsA) == 1

edpA = subs_in_edp(solutionsA[0], dVdrA, second_central_0())

dVdrA_solution = sp.solve(sp.Eq(edpA, 0), dVdrA)

assert len(dVdrA_solution) == 1

dVdrA_value = dVdrA_solution[0]

In [ ]:
# Fluxo continuado

solutions = sp.solve(sp.Eq(dVdrA_value * sig_a, dVdrB_value * sig_b), T_ij)

assert len(solutions) == 1

sp.collect(sp.factor(solutions[0]), [T_ipj, T_imj, T_ijp, T_ijm])

(T_{i+1,j}*(2*\Delta_\phi**2*\Delta_r*\sigma_A*r_{ij}**2 + 4*\Delta_\phi**2*\sigma_A*r_{ij}**3) + T_{i,j+1}*(\Delta_r**3*\sigma_A - \Delta_r**3*\sigma_B + 2*\Delta_r**2*\sigma_A*r_{ij} + 2*\Delta_r**2*\sigma_B*r_{ij}) + T_{i,j-1}*(\Delta_r**3*\sigma_A - \Delta_r**3*\sigma_B + 2*\Delta_r**2*\sigma_A*r_{ij} + 2*\Delta_r**2*\sigma_B*r_{ij}) + T_{i-1,j}*(-2*\Delta_\phi**2*\Delta_r*\sigma_B*r_{ij}**2 + 4*\Delta_\phi**2*\sigma_B*r_{ij}**3))/(2*(\Delta_\phi**2*r_{ij}**2 + \Delta_r**2)*(\Delta_r*\sigma_A - \Delta_r*\sigma_B + 2*\sigma_A*r_{ij} + 2*\sigma_B*r_{ij}))

# Rosa


In [ ]:
# Dado no enunciado

# Roxo

Análogo ao anterior, mas para $\phi$

1. Taylor regressivo em B

2. Taylor progressivo em A

3. Igualar equações

**Obs:** ocorreu uma corrreção do enunciado $\partial V / \partial n \ne 0$ es for contorno interno

In [ ]:
# Desenvolvimento em B

taylor_exp_B, dVd0B, d2Vd02B = taylor_0(material='B', which='reg')

d2Vd02_solutions_B = sp.solve(sp.Eq(T_ijm, taylor_exp_B), d2Vd02B)

assert len(d2Vd02_solutions_B) == 1

d2Vd02_value_B = d2Vd02_solutions_B[0]

edpB = subs_in_edp(second_central_r(), first_central_r(), d2Vd02_value_B)

dVd0B_solution = sp.solve(sp.Eq(edpB, 0), dVd0B)

assert len(dVd0B_solution) == 1

dVd0B_value = dVd0B_solution[0]

In [ ]:
# Desenvolvimento em A

taylor_exp_A, dVd0A, d2Vd02A = taylor_0(material='A', which='prog')

d2Vd02_solutions_A = sp.solve(sp.Eq(taylor_exp_A, T_ijp), d2Vd02A)

assert len(d2Vd02_solutions_A) == 1

d2Vd02_value_A = d2Vd02_solutions_A[0]

edpA = subs_in_edp(second_central_r(), first_central_r(), d2Vd02_value_A)

dVd0A_solution = sp.solve(sp.Eq(edpA, 0), dVd0A)

assert len(dVd0A_solution) == 1

dVd0A_value = dVd0A_solution[0]

In [ ]:
# Fluxo continuado

solutions = sp.solve(sp.Eq(dVd0A_value * sig_a, dVd0B_value * sig_b), T_ij)

assert len(solutions) == 1

sp.collect(sp.factor(solutions[0]), [T_ipj, T_imj, T_ijp, T_ijm])

(T_{i+1,j}*(\Delta_\phi**2*\Delta_r*\sigma_A*r_{ij} + \Delta_\phi**2*\Delta_r*\sigma_B*r_{ij} + 2*\Delta_\phi**2*\sigma_A*r_{ij}**2 + 2*\Delta_\phi**2*\sigma_B*r_{ij}**2) + 4*T_{i,j+1}*\Delta_r**2*\sigma_A + 4*T_{i,j-1}*\Delta_r**2*\sigma_B + T_{i-1,j}*(-\Delta_\phi**2*\Delta_r*\sigma_A*r_{ij} - \Delta_\phi**2*\Delta_r*\sigma_B*r_{ij} + 2*\Delta_\phi**2*\sigma_A*r_{ij}**2 + 2*\Delta_\phi**2*\sigma_B*r_{ij}**2))/(4*(\sigma_A + \sigma_B)*(\Delta_\phi**2*r_{ij}**2 + \Delta_r**2))

# Laranja

1. Expandir por Taylor regressivo até segunda ordem para $\phi$

2. Utilizar a condição de contorno dada no enunciado: $\partial V / \partial \phi = 0$

3. Substituir na EDP

In [ ]:
taylor_exp, dVd0, d2Vd02 = taylor_0(material='', which='reg')

taylor_0_eqn = sp.Eq(T_ijm, taylor_exp.subs({dVd0 : 0}))

d2Vd02_solutions = sp.solve(taylor_0_eqn, d2Vd02)

assert len(d2Vd02_solutions) == 1

d2Vd02_value = d2Vd02_solutions[0]

edp = subs_in_edp(second_central_r(), first_central_r(), d2Vd02_value)

solutions = sp.solve(sp.Eq(edp, 0), T_ij)

assert len(solutions) == 1

sp.collect(sp.factor(solutions[0]), [T_ipj, T_imj, T_ijp, T_ijm])

(T_{i+1,j}*(\Delta_\phi**2*\Delta_r*r_{ij} + 2*\Delta_\phi**2*r_{ij}**2) + 4*T_{i,j-1}*\Delta_r**2 + T_{i-1,j}*(-\Delta_\phi**2*\Delta_r*r_{ij} + 2*\Delta_\phi**2*r_{ij}**2))/(4*(\Delta_\phi**2*r_{ij}**2 + \Delta_r**2))